In [1]:
import pandas as pd
import numpy as np

In [ ]:
data_git = pd.read_csv(r"E:\data\merge_commit.csv", sep=",", header=None)
data_git.columns = ["id", "commit_hash", "is_conflicting", "p1", "p2", "project_id", "is_done", "author_name", 
                 "author_email", "timestamp"]

In [ ]:
results_git = pd.read_csv(r"E:\data\manual_analysis_results.csv", sep=";")

In [ ]:
results_git["project_id"].value_counts().count()

In [ ]:
refactoring_region = pd.read_csv(r"E:\data\refactoring_region.csv", sep=",", header=None)
refactoring_region.columns = ["id", "start_line", "length", "type", "path", "refactoring_id", "refactoring_commit_id", 
                              "commit_hash", "project_id"]

In [ ]:
conflicting_region_history = pd.read_csv(r"E:\data\conflicting_region_history.csv", sep=",", header=None)
conflicting_region_history.columns = ["id", "commit_hash", "merge_parent", "old_start_line", "old_length", "old_path", 
                                      "new_start_line", "new_length", "new_path", "conflicting_region_id", 
                                      "conflicting_java_file_id", "merge_commit_id", "project_id", "author_name", 
                                      "author_email", "timestamp"]

In [ ]:
conflicting_region_history.drop(columns = ["old_start_line", "old_length", "old_path", "new_start_line", "new_length", 
                                           "new_path", "author_name", "author_email", "conflicting_region_id", 
                                           "conflicting_java_file_id", "conflicting_region_id"], inplace=True)

In [ ]:
evolutionary_commits_with_refactoring = refactoring_region.merge(conflicting_region_history, on="commit_hash", how="inner")

In [ ]:
chr_and_refactorings = refactoring_region.merge(conflicting_region_history, on="commit_hash", how="right")

In [ ]:
chr_and_refactorings.drop_duplicates(subset="id_y", inplace=True)

In [ ]:
data = chr_and_refactorings
df = pd.DataFrame(data)

condition = df.isna().any(axis=1)
regs_with_nan = df[condition]

# evolutionary and without refactoring
print("Registers containing at least one NaN:")
display(regs_with_nan)

In [ ]:
chr_rr = regs_with_nan.drop(columns=["id_x", "start_line", "length", "type", "path", "refactoring_id", 
                                         "refactoring_commit_id", "project_id_x"])

In [ ]:
chr_rr_intersect = conflicting_region_history.merge(refactoring_region, on="commit_hash", how="inner")

In [ ]:
chr_rr_intersect.drop_duplicates(subset="id_x", inplace=True)
chr_rr_intersect.reset_index(drop=True, inplace=True)

In [ ]:
chr_rr_intersect["refactoring"] = 1
chr_rr_intersect = chr_rr_intersect[["commit_hash", "merge_parent", "merge_commit_id", "project_id_y", "t

In [ ]:
chr_rr_intersect.reset_index(drop=True, inplace=True)

In [ ]:
chr_rr["refactoring"] = 0
chr_without_refactoring = chr_rr[["commit_hash", "merge_parent", "merge_commit_id", "project_id_y", "timestamp", "refactoring"]]

In [ ]:
chr_rr_intersect

In [6]:
chr_with_without_refactoring = pd.read_csv(r"E:\data\evolutionary_with_without_refactorings.csv", sep=",")



In [8]:
def return_pairs(df_left) :
    pairs = pd.DataFrame(columns = ["commit_hash_origin", "commit_hash_destiny", "project_id", "merge_commit_id"])
    df_all = pd.DataFrame()
    for i in range(1, len(df_left)):
        current_line = df_left.iloc[i]  
        previous_line = df_left.iloc[i - 1]  
        df = pd.DataFrame()
        if (line['refactoring'] == 1 and previous_line['refactoring'] == 0) or (current_line['refactoring'] == 1 and previous_line['refactoring'] == 1):
            x = previous_line.commit_hash
            y = current_line.commit_hash
            z = current_line.merge_commit_id
            w = current_line.project_id_y
            data2 = {'commit_hash_origin': x, 'commit_hash_destiny': y, 'project_id': w, 'merge_commit_id': z}
            #print(data2)
            new_line = pd.Series(data2, name=len(df))
            df = df.append(new_line)
            pairs = pairs.append(df, ignore_index=True)
            df_refactorings = evolutionary_refactorings(aux)
            df_all = df_all.append(df_refactorings, ignore_index=True)

    return pairs, df_all

In [9]:
def evolutionary_refactorings(aux):
    df = pd.DataFrame()
    aux.reset_index(drop=True, inplace=True)
    if len(aux) == aux["refactoring"].sum():
        data2 = {'project_id': aux["project_id_y"][0], 'merge_commit_id': aux["merge_commit_id"][0], 'all_commits_ref': 1}
        new_line = pd.Series(data2, name=len(df))
        df = df.append(new_line)      
    else :
        data2 = {'project_id': aux["project_id_y"][0], 'merge_commit_id': aux["merge_commit_id"][0], 'all_commits_ref': 0}
        new_line = pd.Series(data2, name=len(df))
        df = df.append(new_line)
    return df

In [ ]:
result = pd.DataFrame()
refs = pd.DataFrame()
for i in chr_with_without_refactoring.merge_commit_id.unique() :
    aux = chr_with_without_refactoring[chr_with_without_refactoring["merge_commit_id"] == i].drop_duplicates("commit_hash")
    df_left = aux.query("merge_parent == 1").reset_index(drop=True)
    df_right = aux.query("merge_parent == 2").reset_index(drop=True)
    pairs_left , refactorings_evolutionary_left = return_pairs(df_left)
    pairs_right , refactorings_evolutionary_right = return_pairs(df_right)
    result = result.append(pairs_left, ignore_index=True)
    result = result.append(pairs_right, ignore_index=True)
    refs = refs.append(refactorings_evolutionary_left, ignore_index=True)
    refs = refs.append(refactorings_evolutionary_right, ignore_index=True)

In [ ]:
# Collecting a random sample
sample_size = 96
random_sample = np.random.choice(result.index, sample_size, replace=False)

print("Random sample:", random_sample)

In [15]:
ordered_numbers = sorted(random_sample)

In [ ]:
df = pd.DataFrame()
for id in ordered_numbers:
    data2 = {'commit_hash_origin': result["commit_hash_origin"][id], 'commit_hash_destiny': result["commit_hash_destiny"][id], 
             'project_id': result["project_id"][id], 'merge_commit_id': result["merge_commit_id"][id]}
    #print(data2)
    mew_line = pd.Series(data2, name=len(df))
    df = df.append(new_line)

In [ ]:
df.to_csv("manual_test_pairs_evol_commits.csv")